In [1]:
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline, PipelineModel


In [2]:
df = spark.createDataFrame(
     [(0, 0, 2.0), (0, 1, 2.0), (0, 2, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0),(3, 0, 2.0), (3, 1, 2.0),(4, 0, 2.0), (4, 1, 2.0), (4, 2, 2.0)],
     ["user", "item", "rating"])

In [3]:
continuous_feature_assembler= VectorAssembler(inputCols=['user','item','rating'], outputCol="unscaled_continuous_features")

In [4]:
estimatorsLr = [continuous_feature_assembler]

In [5]:
featurePipeline = Pipeline(stages=estimatorsLr)

In [6]:
sparkFeaturePipelineModel = featurePipeline.fit(df)

In [7]:
sparkFeaturePipelineModel.transform(df).show()

+----+----+------+----------------------------+
|user|item|rating|unscaled_continuous_features|
+----+----+------+----------------------------+
|   0|   0|   2.0|               [0.0,0.0,2.0]|
|   0|   1|   2.0|               [0.0,1.0,2.0]|
|   0|   2|   2.0|               [0.0,2.0,2.0]|
|   1|   1|   3.0|               [1.0,1.0,3.0]|
|   1|   2|   4.0|               [1.0,2.0,4.0]|
|   2|   1|   1.0|               [2.0,1.0,1.0]|
|   2|   2|   5.0|               [2.0,2.0,5.0]|
|   3|   0|   2.0|               [3.0,0.0,2.0]|
|   3|   1|   2.0|               [3.0,1.0,2.0]|
|   4|   0|   2.0|               [4.0,0.0,2.0]|
|   4|   1|   2.0|               [4.0,1.0,2.0]|
|   4|   2|   2.0|               [4.0,2.0,2.0]|
+----+----+------+----------------------------+



In [9]:
%savePipeline   --pipelineName testassembler \
                --pipelineObject sparkFeaturePipelineModel \
                --dataframe df \
                --pipelineFilename testAssembler \
                --description "Test Vector Assembler"

{"message":"Model 'testassembler' correctly uploaded."}


In [12]:
%deployPipeline --pipelineName testassembler

Error while deploying 'testassembler' model from repository.  Response from server:
 {"error":"MarathonException","reason":"Error in Marathon. Code: 401  Body: <!DOCTYPE html>\n\n<html>\n<head>\n  <title>Unauthorized</title>\n  <noscript><meta http-equiv=\"refresh\" content=\"5; url=/#/login\"></noscript>\n  <script>\n    (function () {\n      var location = window.location;\n      location.href = \"/#/login?redirect=\" + encodeURIComponent(location.href);\n    }())\n  </script>\n</head>\n<body>\n  <h1>Unauthorized</h1>\n</body>\n</html>\n\n"}


In [19]:
%downloadPipeline --pipelineName testassembler --outDir /var/sds/homes/fdeplasencia/workspace/dojo2

Successful downloaded serialized file 'testAssembler.zip' associated with the 'testassembler' model.


In [17]:
pwd

'/var/sds/homes/fdeplasencia/workspace/bc2_recommendation'

In [20]:
ls ../dojo2

Codeproducts2.csv  Dojo_empty.ipynb  testAssembler.zip  Untitled.ipynb


In [13]:
%listPipelines

[
  {
    "modelName": "pruebaassembler",
    "versions": [
      {
        "id": "36a328d7-108d-4548-8b2e-978152027ac2",
        "serializationLib": "mleap",
        "kernel": "Python3 PySpark 2.1.0",
        "user": "stratio",
        "modelPath": "/ml-models/pruebaassembler/v0/testAssembler.zip",
        "modelDescription": "Test Vector Assembler",
        "modelVersion": 1
      }
    ]
  },
  {
    "modelName": "testassembler",
    "versions": [
      {
        "id": "7b5e3061-3c88-436f-92a1-0a8d3d5f6fdc",
        "serializationLib": "mleap",
        "kernel": "Python3 PySpark 2.1.0",
        "user": "fdeplasencia",
        "modelPath": "/ml-models/testassembler/v0/testAssembler.zip",
        "modelDescription": "Test Vector Assembler",
        "modelVersion": 1
      }
    ]
  }
]


In [4]:
df.show()

+----+----+------+
|user|item|rating|
+----+----+------+
|   0|   0|   4.0|
|   0|   1|   2.0|
|   1|   1|   3.0|
|   1|   2|   4.0|
|   2|   1|   1.0|
|   2|   2|   5.0|
+----+----+------+



In [58]:
als = ALS(rank=10, maxIter=5, seed=0, implicitPrefs=True)

In [59]:
model = als.fit(df)

In [60]:
model.rank

10

In [61]:
model.userFactors.show(truncate=False)

+---+--------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                        |
+---+--------------------------------------------------------------------------------------------------------------------------------+
|0  |[0.06361665, -0.23074257, 0.20969664, 0.1825702, 0.46861342, -0.07362809, -0.006829679, 0.109823756, 0.34183514, -0.32252687]   |
|1  |[0.17628323, -0.12033432, 0.15458374, -0.36101434, 0.14567104, -0.37776175, -0.2297353, 0.06103852, 0.5263435, -0.00200798]     |
|2  |[0.1738038, -0.13651478, 0.16337444, -0.3435888, 0.14367773, -0.37089735, -0.2100658, 0.049358495, 0.52029943, 0.010040237]     |
|3  |[-0.010812119, 0.30984813, -0.15642485, -0.005151795, 0.40158036, 0.042911623, -0.35937357, 0.37866098, 0.081038624, -0.6467516]|
|4  |[0.06361665, -0.23074257, 0.20969664, 0.1825702, 0

In [62]:
model.userFactors.orderBy("id").collect()

[Row(id=0, features=[0.06361664831638336, -0.23074257373809814, 0.20969663560390472, 0.1825702041387558, 0.4686134159564972, -0.07362809032201767, -0.006829679012298584, 0.10982375591993332, 0.3418351411819458, -0.32252687215805054]),
 Row(id=1, features=[0.17628322541713715, -0.12033431977033615, 0.15458373725414276, -0.36101433634757996, 0.1456710398197174, -0.37776175141334534, -0.22973529994487762, 0.06103852018713951, 0.5263435244560242, -0.0020079798996448517]),
 Row(id=2, features=[0.17380380630493164, -0.1365147829055786, 0.16337443888187408, -0.34358879923820496, 0.1436777263879776, -0.3708973526954651, -0.2100657969713211, 0.04935849457979202, 0.5202994346618652, 0.01004023663699627]),
 Row(id=3, features=[-0.010812118649482727, 0.3098481297492981, -0.1564248502254486, -0.005151795223355293, 0.4015803635120392, 0.04291162267327309, -0.3593735694885254, 0.37866097688674927, 0.08103862404823303, -0.6467515826225281]),
 Row(id=4, features=[0.06361664831638336, -0.230742573738098

In [63]:
test = spark.createDataFrame([ (1, 0), (2, 0),(0,0),(3,2)], ["user", "item"])

In [64]:
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])

In [65]:
predictions[0]

Row(user=0, item=0, prediction=0.963922917842865)

In [66]:
predictions[1]

Row(user=1, item=0, prediction=0.09879466891288757)

In [67]:
predictions[2]

Row(user=2, item=0, prediction=0.09470575302839279)

In [68]:
als.getMaxIter()

5

In [69]:
model.transform(test).show()

+----+----+----------+
|user|item|prediction|
+----+----+----------+
|   3|   2|0.17343059|
|   1|   0|0.09879467|
|   2|   0|0.09470575|
|   0|   0| 0.9639229|
+----+----+----------+



In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %deletePipeline  %deployPipeline  %dhist  %dirs  %doctest_mode  %downloadPipeline  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %listPipelines  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %savePipeline  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html 

In [3]:
%listPipelines

ConnectionError: HTTPSConnectionPool(host='intellmodelrep1.intelligence.marathon.mesos', port=8000): Max retries exceeded with url: /models (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7fa4919dcc18>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [1]:

data = [["a", "b", "c"], ["a", "b", "d", "e"], ["a", "c", "e"], ["a", "c", "f"]]

In [3]:
rdd = sc.parallelize(data, 2)

In [5]:
rdd.take(5)

[['a', 'b', 'c'], ['a', 'b', 'd', 'e'], ['a', 'c', 'e'], ['a', 'c', 'f']]

In [11]:
model = FPGrowth.train(rdd, 0.6, 2)

In [17]:
from pyspark.mllib.fpm import FPGrowth, FPGrowthModel

In [6]:
model = FPGrowth.train(rdd, 0.6, 2)

In [9]:
model.save(sc,path='/analytics/bc2_recommender/models')

In [18]:
sameModel = FPGrowthModel.load(sc, '/analytics/bc2_recommender/models')

In [21]:
sorted(sameModel.freqItemsets().collect())

[FreqItemset(items=['a'], freq=4),
 FreqItemset(items=['c'], freq=3),
 FreqItemset(items=['c', 'a'], freq=3)]

In [13]:
from pyspark.mllib import FPGrowthModel

SyntaxError: invalid syntax (<ipython-input-13-8cbafdfb2e32>, line 1)

In [6]:
sorted(model.freqItemsets().collect())

[FreqItemset(items=['a'], freq=4),
 FreqItemset(items=['c'], freq=3),
 FreqItemset(items=['c', 'a'], freq=3)]

In [ ]:
model.save()

In [10]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %deletePipeline  %deployPipeline  %dhist  %dirs  %doctest_mode  %downloadPipeline  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %listPipelines  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %savePipeline  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html 